In [ ]:
%pip install evadb
%pip install openai
%pip install unstructured
%pip install langchain
%pip install tiktoken
%pip install streamlit
import evadb
import os

In [5]:
%pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following versions that require a different python version: 0.1.1 Requires-Python >=3.9; 0.1.2 Requires-Python >=3.8; 0.2.0 Requires-Python >=3.8; 0.3.0 Requires-Python >=3.8; 0.3.1 Requires-Python >=3.8; 0.3.2 Requires-Python >=3.8; 0.3.3 Requires-Python >=3.8; 0.4.0 Requires-Python >=3.8; 0.5.0 Requires-Python >=3.8; 0.5.1 Requires-Python >=3.8
ERROR: Could not find a version that satisfies the requirement tiktoken (from versions: none)
ERROR: No matching distribution found for tiktoken


In [2]:

import os

In [3]:
print(os.__version__)

AttributeError: module 'os' has no attribute '__version__'

In [6]:
import streamlit as st, tiktoken
from langchain.chat_models import ChatOpenAI
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains.summarize import load_summarize_chain

ModuleNotFoundError: No module named 'tiktoken'

In [3]:
cursor = evadb.connect().cursor()

Downloading: "http://ml.cs.tsinghua.edu.cn/~chenxi/pytorch-models/mnist-b07bb66b.pth" to /root/.cache/torch/hub/checkpoints/mnist-b07bb66b.pth
100%|██████████| 1.03M/1.03M [00:01<00:00, 875kB/s]
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


In [ ]:
print(cursor.query(
    """CREATE TABLE IF NOT EXISTS News111 (title1 TEXT(50), link TEXT(50), summary TEXT(50));"""
).execute())
cursor.query("INSERT INTO News111 (title1, link, summary) VALUES ('1', '2', '3')").df();
cursor.query("SELECT * FROM News111").df()
SerperAPI = str(input(
            "Please enter your Serper API key: "))
url = 'https://api.example.com/data'
OpenAPI = str(
        input(
            "Please enter your OpenAI API key: "
        )
    )
keyWord = str(
    input(
        "Please enter your key word: "
    )
)



os.environ["SERPER_KEY"] = SerperAPI
os.environ["OPEN_KEY"] = OpenAPI

In [ ]:
search = GoogleSerperAPIWrapper(type="news", tbs="qdr:w1", serper_api_key=SerperAPI)
cursor.drop_table("News111", if_exists=True).execute()
result_dict = search.results("word")
print(cursor.query(
    """CREATE TABLE IF NOT EXISTS News111 (title1 TEXT(50), link TEXT(50), summary TEXT(50));"""
).execute())
cursor.query("SELECT * FROM News111").df()
count = 0
for news_item in result_dict['news']:
    if count < 3:
        title = news_item['title'][:50]  # Limit the title to 50 characters
        link = news_item['link']    # Limit the link to 50 characters
        loader = UnstructuredURLLoader(urls=[news_item['link']])
        data = loader.load()
        llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo', openai_api_key=OpenAPI)
        chain = load_summarize_chain(llm, chain_type="map_reduce")
        summary = chain.run(data)
        try:
            cursor.query(f"INSERT INTO News111 (title1, link, summary) VALUES ('{title}', '{link}', '{summary}')").df();
            cursor.query("SELECT * FROM News111").df()
            count += 1
        except:
            pass
    else:
      break

In [ ]:
cursor.query("SELECT * FROM News111").df()

In [ ]:
# cursor.query("DROP UDF IF EXISTS Similarity;").execute()
Similarity_function_query = """CREATE UDF Similarity
                INPUT (Frame_Array_Open NDARRAY UINT8(3, ANYDIM, ANYDIM),
                        Frame_Array_Base NDARRAY UINT8(3, ANYDIM, ANYDIM),
                        Feature_Extractor_Name TEXT(100))
                OUTPUT (distance FLOAT(32, 7))
                TYPE NdarrayFunction
                IMPL './similarity.py'"""

cursor.query(Similarity_function_query).execute()